In [1]:
%pip install prince

  Obtaining dependency information for prince from https://files.pythonhosted.org/packages/ea/47/05a78e27a6c7f85b5e006169e4ddf27637867124ef841176f5e4f5ce7f88/prince-0.13.0-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6.0.0,>=4.2.2 from https://files.pythonhosted.org/packages/17/16/b12fca347ff9d062e3c44ad9641d2ec50364570a059f3078ada3a5119d7a/altair-5.1.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/55.8 kB ? eta -:--:--
     ----------------------------- ---------- 41.0/55.8 kB 2.0 MB/s eta 0:00:01
     ----------------------------- ---------- 41.0/55.8 kB 2.0 MB/s eta 0:00:01
     ----------------------------- ---------- 41.0/55.8 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 55.8/55.8 kB 292.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/415.6 kB ? eta -:--:--
   --- ------------------------------------ 41.0/415.6 kB 1.9 MB/s eta 0:00:01
   ----- --------------------------------- 61.4/4

In [58]:
import prince
dataset = prince.datasets.load_decathlon()

In [61]:
olympic = dataset.query('competition == "OlympicG"')

In [62]:
print(type(olympic))

<class 'pandas.core.frame.DataFrame'>


In [63]:
olympic.head()

100m  long.jump  shot.put  high.jump   400m  \
competition athlete                                                 
OlympicG    Sebrle   10.85       7.84     16.36       2.12  48.36   
            Clay     10.44       7.96     15.23       2.06  49.19   
            Karpov   10.50       7.81     15.93       2.09  46.81   
            Macey    10.89       7.47     15.73       2.15  48.97   
            Warners  10.62       7.74     14.48       1.97  47.97   

                     110m.hurdle  discus  pole.vault  javeline   1500m  rank  \
competition athlete                                                            
OlympicG    Sebrle         14.05   48.72         5.0     70.52  280.01     1   
            Clay           14.13   50.11         4.9     69.71  282.00     2   
            Karpov         13.97   51.65         4.6     55.54  278.11     3   
            Macey          14.56   48.34         4.4     58.46  265.42     4   
            Warners        14.01   43.73         4.9     55.39  278.05     5   

                     points  
competition athlete          
OlympicG    Sebrle     8893  
            Clay       8820  
            Karpov     8725  
            Macey      8414  
            Warners    8343

In [75]:
pca = prince.PCA(n_components=5)

In [76]:
pca = pca.fit(olympic, supplementary_columns=['rank', 'points'])

In [77]:
pca.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,3.545,35.45%,35.45%
1,1.970,19.70%,55.15%
2,1.422,14.22%,69.36%
3,0.903,9.03%,78.40%
4,0.564,5.64%,84.03%


In [78]:
pca.transform(dataset).head()

component                   0         1         2         3         4
competition athlete                                                  
Decastar    Sebrle   0.447363  0.766253  1.004892  1.224097  0.433196
            Clay     0.810219  0.450651  1.895999 -0.057219 -1.699552
            Karpov   0.908032  0.628018  2.240411 -1.463540  0.307295
            Bernard -0.786697 -1.060280  0.621473  2.175319 -0.263518
            Yurkov  -0.915965  2.218787 -0.889543  0.851939  0.586956

In [79]:
import matplotlib.pyplot as plt
chart = pca.plot(dataset)

In [80]:
chart

alt.LayerChart(...)

In [81]:
chart = pca.plot(
     dataset,
     show_row_labels=True,
     show_row_markers=False,
     row_labels_column='athlete',
     color_rows_by='competition'
 )

In [82]:
chart

alt.LayerChart(...)

In [72]:
chart = pca.plot(
     dataset,
     show_row_labels=True,
     show_row_markers=True,
     row_labels_column='athlete',
     color_rows_by='competition'
 )

In [73]:
chart

alt.LayerChart(...)

In [83]:
dataset = prince.datasets.load_burgundy_wines()
dataset

expert Oak type Expert 1               Expert 2                         \
aspect            Fruity Woody Coffee Red fruit Roasted Vanillin Woody   
Wine 1        1        1     6      7         2       5        7     6   
Wine 2        2        5     3      2         4       4        4     2   
Wine 3        2        6     1      1         5       2        1     1   
Wine 4        2        7     1      2         7       2        1     2   
Wine 5        1        2     5      4         3       5        6     5   
Wine 6        1        3     4      4         3       5        4     5   

expert Expert 3               
aspect   Fruity Butter Woody  
Wine 1        3      6     7  
Wine 2        4      4     3  
Wine 3        7      1     1  
Wine 4        2      2     2  
Wine 5        2      6     6  
Wine 6        1      7     5

In [84]:
groups = dataset.columns.levels[0].drop("Oak type").tolist()
groups

['Expert 1', 'Expert 2', 'Expert 3']

In [85]:
mfa = prince.MFA(
    n_components=2,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
mfa = mfa.fit(dataset, groups=groups)

In [86]:
mfa.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,2.835,88.82%,88.82%
1,0.357,11.18%,100.00%


In [87]:
mfa.row_coordinates(dataset)

,0,1
Wine 1,-2.172155,-0.508596
Wine 2,0.557017,-0.197408
Wine 3,2.317663,-0.830259
Wine 4,1.832557,0.905046
Wine 5,-1.403787,0.054977
Wine 6,-1.131296,0.576241


In [88]:
mfa.group_row_coordinates(dataset)

group      Expert 1            Expert 2            Expert 3          
component         0         1         0         1         0         1
Wine 1    -2.764432 -1.104812 -2.213928 -0.863519 -1.538106  0.442545
Wine 2     0.773034  0.298919  0.284247 -0.132135  0.613771 -0.759009
Wine 3     1.991398  0.805893  2.111508  0.499718  2.850084 -3.796390
Wine 4     1.981456  0.927187  2.393009  1.227146  1.123206  0.560803
Wine 5    -1.292834 -0.620661 -1.492114 -0.488088 -1.426414  1.273679
Wine 6    -0.688623 -0.306527 -1.082723 -0.243122 -1.622541  2.278372

In [89]:
mfa.plot(
    dataset,
    x_component=0,
    y_component=1
)

alt.Chart(...)

In [90]:
dataset['Expert 1']

aspect,Fruity,Woody,Coffee
Wine 1,1,6,7
Wine 2,5,3,2
Wine 3,6,1,1
Wine 4,7,1,2
Wine 5,2,5,4
Wine 6,3,4,4


In [91]:
mfa['Expert 1'].eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,2.863,95.42%,95.42%
1,0.120,3.99%,99.41%


In [92]:
dataset = prince.datasets.load_energy_mix(year=2019, normalize=True)
dataset.head().style.format('{:.0%}')  

,,coal,oil,gas,nuclear,hydro,wind,solar,other renewables
continent,country,,,,,,,,
Africa,Algeria,1%,35%,64%,0%,0%,0%,0%,0%
South America,Argentina,1%,35%,50%,2%,10%,1%,0%,1%
Oceania,Australia,28%,34%,30%,0%,2%,3%,3%,1%
Europe,Austria,9%,37%,22%,0%,25%,4%,1%,3%
Asia,Azerbaijan,0%,33%,65%,0%,2%,0%,0%,0%


In [93]:
pca = prince.PCA(
    n_components=3,
    n_iter=3,
    rescale_with_mean=True,
    rescale_with_std=True,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
pca = pca.fit(dataset)

In [94]:
pca.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,1.963,24.54%,24.54%
1,1.561,19.51%,44.05%
2,1.403,17.54%,61.59%


In [95]:
pca.eigenvalues_

array([1.96301214, 1.56069986, 1.40327823])

In [96]:
pca.percentage_of_variance_

array([24.5376518 , 19.50874822, 17.54097785])

In [97]:
pca.cumulative_percentage_of_variance_

array([24.5376518 , 44.04640003, 61.58737788])

In [98]:
pca.scree_plot()

alt.Chart(...)

In [99]:
pca.transform(dataset).head()

,component,0,1,2
continent,country,,,
Africa,Algeria,-2.189068,0.380243,-0.388572
South America,Argentina,-1.244981,0.801917,-0.389456
Oceania,Australia,0.203969,-1.470254,0.531819
Europe,Austria,0.847122,1.008296,-0.521998
Asia,Azerbaijan,-2.190535,0.632250,-0.365515


In [100]:
pca.transform(dataset).equals(pca.row_coordinates(dataset))

True

In [101]:
pca.column_coordinates_

component,0,1,2
variable,,,
coal,0.230205,-0.395474,0.740748
oil,0.176200,-0.418917,-0.737855
gas,-0.866927,0.182990,-0.096857
nuclear,0.310313,-0.002598,0.400608
hydro,0.440383,0.744815,-0.016375
wind,0.518712,-0.161507,-0.364593
solar,0.415677,-0.589288,0.001012
other renewables,0.628750,0.516935,-0.084114


In [102]:
pca.plot(
    dataset,
    x_component=0,
    y_component=1,
    color_rows_by='continent',
    show_row_markers=True,
    show_column_markers=True,
    show_row_labels=False,
    row_labels_column=None,  # for DataFrames with a MultiIndex
    show_column_labels=False
)

alt.LayerChart(...)

In [103]:
pca.row_contributions_.head().style.format('{:.0%}')  

,component,0,1,2
continent,country,,,
Africa,Algeria,3%,0%,0%
South America,Argentina,1%,1%,0%
Oceania,Australia,0%,2%,0%
Europe,Austria,0%,1%,0%
Asia,Azerbaijan,3%,0%,0%


In [104]:
pca.column_contributions_.style.format('{:.0%}')

component,0,1,2
variable,,,
coal,3%,10%,39%
oil,2%,11%,39%
gas,38%,2%,1%
nuclear,5%,0%,11%
hydro,10%,36%,0%
wind,14%,2%,9%
solar,9%,22%,0%
other renewables,20%,17%,1%


In [105]:
pca.row_cosine_similarities(dataset).head()

,component,0,1,2
continent,country,,,
Africa,Algeria,0.902764,0.027238,0.028445
South America,Argentina,0.626517,0.259936,0.061309
Oceania,Australia,0.008468,0.439999,0.057570
Europe,Austria,0.236087,0.334470,0.089643
Asia,Azerbaijan,0.868801,0.072377,0.024190


In [106]:
pca.column_cosine_similarities_

component,0,1,2
variable,,,
coal,0.052994,0.156400,0.548707
oil,0.031046,0.175492,0.544430
gas,0.751563,0.033485,0.009381
nuclear,0.096294,0.000007,0.160487
hydro,0.193937,0.554750,0.000268
wind,0.269062,0.026085,0.132928
solar,0.172788,0.347260,0.000001
other renewables,0.395327,0.267222,0.007075


In [107]:
pca.column_correlations

component,0,1,2
variable,,,
coal,0.230205,-0.395474,0.740748
oil,0.176200,-0.418917,-0.737855
gas,-0.866927,0.182990,-0.096857
nuclear,0.310313,-0.002598,0.400608
hydro,0.440383,0.744815,-0.016375
wind,0.518712,-0.161507,-0.364593
solar,0.415677,-0.589288,0.001012
other renewables,0.628750,0.516935,-0.084114


In [108]:
(pca.column_correlations ** 2).equals(pca.column_cosine_similarities_)

True

In [109]:
reconstructed = pca.inverse_transform(pca.transform(dataset))
reconstructed.head()

,,0,1,2,3,4,5,6,7
continent,country,,,,,,,,
Africa,Algeria,0.024204,0.369866,0.599244,-0.007082,0.024826,-0.003249,0.000650,-0.008460
South America,Argentina,0.027773,0.366342,0.487562,0.008596,0.090082,0.006963,0.001391,0.011292
Oceania,Australia,0.287372,0.425348,0.208594,0.056641,-0.018484,0.025867,0.015104,-0.000441
Europe,Austria,0.060875,0.410393,0.219553,0.040122,0.183231,0.034237,0.006167,0.045423
Asia,Azerbaijan,0.013098,0.354083,0.606928,-0.006558,0.042620,-0.004640,-0.000439,-0.005092


In [110]:
import numpy as np

np.linalg.norm(reconstructed.values - dataset.values, ord=2)

1.192715958987242

In [111]:
active = dataset.query("continent != 'North America'")
pca = prince.PCA().fit(active)

In [112]:
pca.transform(dataset).query("continent == 'North America'")

component                                 0         1
continent     country                                
North America Canada               0.009864  1.332859
              Mexico              -0.695023 -0.402230
              Trinidad and Tobago -3.072833  1.393067
              United States       -0.226122 -0.433260

In [113]:
pca = prince.PCA().fit(dataset, supplementary_columns=['hydro', 'wind', 'solar'])
pca.column_correlations

component,0,1
variable,,
coal,0.621633,-0.380008
oil,0.048966,0.948650
gas,-0.916078,-0.331546
nuclear,0.384458,-0.401675
other renewables,0.467116,0.086656
hydro,0.246646,0.037118
wind,0.195675,0.184247
solar,0.251247,0.076237


In [114]:
pca = prince.PCA().fit(active, supplementary_columns=['hydro', 'wind', 'solar'])

In [115]:
prince.PCA(engine='fbpca')

PCA(engine='fbpca')

In [116]:
prince.PCA(engine='scipy')

PCA(engine='scipy')

In [117]:
import itertools
import numpy as np
import pandas as pd

N = [100, 10_000, 100_000]
P = [3, 10, 30]
ENGINES = ['sklearn', 'fbpca', 'scipy']

perf = []

In [119]:
%pip install fbpca 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11390 sha256=e12fcc125651eb85ed41efe72874eb6535c0eb60f12c1674d630b83085749c6e
  Stored in directory: c:\users\sures\appdata\local\pip\cache\wheels\b4\3b\77\a06a07a415b222f47a7e522333f85ce64c0defd07a57762267
Successfully built fbpca
Note: you may need to restart the kernel to use updated packages.


In [121]:
import fbpca

for n, p, engine in itertools.product(N, P, ENGINES):

    # Too slow
    if engine == 'scipy' and n > 10_000:
        continue

    X = pd.DataFrame(np.random.random(size=(n, p)))
    duration = %timeit -q -n 1 -r 3 -o prince.PCA(engine=engine, n_iter=3).fit(X)
    perf.append({
        'n': n,
        'p': p,
        'engine': engine,
        'seconds': duration.average
    })

ValueError: fbpca is not installed; please install it if you want to use it

In [ ]:
(
    pd.DataFrame(perf)
    .set_index(['n', 'p'])
    .groupby(['n', 'p'], group_keys=False)
    .apply(lambda x: x[['engine', 'seconds']]
    .sort_values('seconds'))
)